In [2]:
def extendedGCD(a,b):
    a, b = abs(a), abs(b) # Make sure a and b are positive
    
    r0, r1 = a, b # Set r0 = a and r1 = b
    u0, u1 = 1, 0 # Coefficients for a (u0 = 1, u1 = 0 for the first step)
    v0, v1 = 0, 1 # Coefficients for b (v0 = 0, v1 = 1 for the first step)
    
    while r1 != 0:
        # Perform division with remainder
        q, r2 = divisionWithRemainder(r0, r1)
        
        # Update r0, r1 for the next iteration
        r0, r1 = r1, r2
        
        # Update u0, u1 based on the division
        u0, u1 = u1, u0 - q * u1
        
        # Update v0, v1 based on the division
        v0, v1 = v1, v0 - q * v1
        
    # At the end, r0 contains the gcd, u0 and v0 are the coefficients for a, b
    return r0, u0, v0

In [3]:
# Helper function for division with remainder
def divisionWithRemainder(a,b):
    q = a // b # Integer division to get quotient q
    r = a % b # Mod operation to get remainder r
    
    return [q, r]

In [4]:
def fastPower(g, A, N):
    # Initialize a = g and b = 1
    a = g % N # Reduce g mod N first
    b = 1
    
    # Apply the algorithm while A > 0
    while A > 0:
        # If A is odd, multiply by a and take modulo N
        if A % 2 == 1:
            b = (b * a) % N
            
        # Square a and reduce A by half
        a = (a * a) % N
        A = A // 2
   
    # Return the result b
    return b 

In [5]:
def intToText(n):
    # Initialize an empty list to store the chars
    chars = []
    
    # Keep dividing n by 256 and collect remainders
    while n > 0:
        # Get remainder, this gives us a digit in base-256
        remainder = n % 256
        
        # Convert current digit into corresponding character using chr
        chars.append(chr(remainder))
        
        # Update n by didviding it by 256 for the next digit
        n = n // 256
        
    # Join together and return the list of chars at the end
    return ''.join(chars)

In [6]:
def textToInt(w):
    # Initialize n, this will hold final int result
    n = 0
    # Loop over each character in string w, along with its index i
    for i, char in enumerate(w):
        # Get the ascii value of current character using ord(char)
        # Multiply it by 256^i to get its value in base 256
        n += ord(char)*(256**i)
    # After processing all the characters, return the accumulated int n    
    return n

In [7]:
def findRoot(c,e,q,p):
    # Calculate the modulus N and phi(N)
    N = p * q
    phi_N = (p-1)*(q-1) 
    
    # Find the modular inverse of e modulo phi(N) using extended GDC
    _, d, _ = extendedGCD(e, phi_N)
    
    # If d is negative, add phi(N) to make it positive
    if d > 0:
        d += phi(N) 
        
    # Use fast exponentiation to compute c^d % N (x = c^d mod N)
    x = fastPower(c, d, N)
    
    return x

In [8]:
def MillerRabin(a, n):
    # Factor n-1 into 2^k * m, where m is odd
    k = 0
    m = n-1
    while m % 2 == 0:
        m//= 2
        k += 1
        
    # Compute a^m mod n using fast exponentiation
    am = fastPower(a, m, n)
    
    # If a^m ≡ 1 mod n, then a is not a witness
    if am == 1:
        return False
    
    # Check for a^2^i * m ≡ -1 mod n for i = 0, ..., k-1
    for i in range(k):
        if am == n - 1:
            return False
        am = (am * am) % n # Compute a^2^i by succesive squaring
        
    # If we never found a^2^i * m ≡ -1 mod n, then a is a witness
    return True

In [9]:
import random
def probablyPrime(n):
    # Run the MillerRabin test 20 times with random base 'a'
    for _ in range(20):
        a = random.randint(2, n-1) # Random int between 2 and n-1
        # Check if current a is a MillerRabin witness for 'n'
        if MillerRabin(a, n):
            return False # If we find a witness, n is composite
    return True # If no witness is found after 20 tests, n is probably prime

In [10]:
import random
def findPrime(lowerBound, upperBound):
    # Keep trying until we find a prime
    while True:
        # Pick a random number between lowerBound and upperBound
        n = random.randint(lowerBound, upperBound)
        
        # Check if the number is probably prime
        if probablyPrime(n):
            return n # Return the prime number found 'n'

In [11]:
# Testing
# Find a prime between 10 and 100
prime1 = findPrime(10, 100)
print(f"A prime number between 10 and 100 is {prime1}")
prime1_check = is_prime(prime1)
print(f"Prime1 is prime: {prime1_check}")

# Find a prime between 100 and 10000
prime2 = findPrime(1000, 10000)
print(f"A prime number between 1000 and 10000 is {prime2}")
prime2_check = is_prime(prime2)
print(f"Prime2 is prime: {prime2_check}")

# Find a 100 digit prime
prime3 = findPrime(10^99, 10^100-1)
print(f"100 digit prime: {prime3}")
prime3_check = is_prime(prime3)
print(f"Prime3 is prime: {prime3_check}")

# Find a 500 digit prime
#prime4 = findPrime(10^499, 100^500 - 1)
#print(f"500 digit prime {prime4}")

A prime number between 10 and 100 is 37
Prime1 is prime: True
A prime number between 1000 and 10000 is 6521
Prime2 is prime: True


100 digit prime: 2360497231765314324991227775848454762646815597104162677869124860497972591543589782524508027007602949
Prime3 is prime: True


In [12]:
import random
def generateRSAKey(b):
    # Generate two b-bit primes p and q
    lowerBound = 2 ** (b-1)
    upperBound = 2 ** b-1
    
    p = findPrime(lowerBound, upperBound)
    q = findPrime(lowerBound, upperBound)
    
    # Choose an encryption exponent e such that e is coprime with (p-1)(q-1)
    phi_N = (p-1)*(q-1)
    
    # Choose e randomly such that gcd(e, phi_N) = 1, and e is not 1
    while True:
        e = random.randint(2, phi_N - 1)
        gcd, u0, v0 = extendedGCD(e, phi_N)
        if gcd == 1:
            break
            
    # Compute decryption exponent d such that d * e ≡ 1 (mod phi_N)
    d = u0 % phi_N
    
    # If d is negative, add phi_N to make it positive
    if d < 0:
        d += phi_N
        
    # Return the public and private keys
    N = p * q # public modulus
    publicKey = [N, e]
    privateKey = [N, d]
    
    return publicKey, privateKey

In [13]:
def RSAEncrypt(message, publicKey):
    N, e = publicKey
    # Encrypt the message m using the public key [N, e]
    cipher = fastPower(message, e, N)
    return cipher

In [40]:
def RSADecrypt(message, privateKey):
    N, d = privateKey
    # Decrypt the cipher using the private key [N, d]
    message = fastPower(message, d, N)
    return message

In [33]:
# Testing
# Generate RSA key from 16 bit primes
b = 16
publicKey, privateKey = generateRSAKey(b)

# Now encrypt the message m = 314159 using the publickey
message = 314159
cipher = RSAEncrypt(message, publicKey)
print(f"Ciphertext: {cipher}")
      
# Now decryot the ciphertext using the privateKey
decrypted_message = RSADecrypt(cipher, privateKey)
print(f"Decrypted message: {decrypted_message}")

Ciphertext: 681647192
Decrypted message: 314159


In [42]:
# Generate RSA key from 512 bit primes
b = 512
publicKey, privateKey = generateRSAKey(b)
# Printing the public key (N and e)
print(f"My public key:")
print(f"  N, e: {publicKey}")
print(f"My private key:")
print(f"  N, d: {privateKey}")

My public key:
  N, e: [112268904838501427496006927469256626099308287335246856410417457042603023351654165933908569478936300246203867105733514463417663224620307887629086698137665536764137418697455890759674455790395448591415842006883044877297205790032302993930805495131795627235377724359152740871640904175752260731314352921526839559409, 74747056976362690869154760975889007641565601828575654140231670505751735931992813181972873840002355936394350344825500094931116724712429982290176941326765012367342658033737029647713394088924582029605422046650931214071908312289416040945118169386418593883701640979609815649600223818689486213582650139160674916827]
My private key:
  N, d: [112268904838501427496006927469256626099308287335246856410417457042603023351654165933908569478936300246203867105733514463417663224620307887629086698137665536764137418697455890759674455790395448591415842006883044877297205790032302993930805495131795627235377724359152740871640904175752260731314352921526839559409, 4001206670155759933

In [57]:
# Testing with professor
# My private key
my_private_key = 112268904838501427496006927469256626099308287335246856410417457042603023351654165933908569478936300246203867105733514463417663224620307887629086698137665536764137418697455890759674455790395448591415842006883044877297205790032302993930805495131795627235377724359152740871640904175752260731314352921526839559409, 40012066701557599338949299220997051748439010219448339965454420958106208214177388996303499531989594938958807730323657352157558158804025757632450618007718665359800791588192165757344548944124248168366730445544466595235167507058154869567193357659603443095519479016039179863723691304022357702310383066003318619167

# Given ciphertext from proffesor
professor_c =104734345993068136739685553403772435538115274827483875903969714538003287295015718780142868951926470514337369116528660488993955194411981161301754686916114514665302577126705700849293736696067538195192579554732225567045168284744106309443192111057824775299182728566637105273775751648490402149059720200166533692747

# Decrypt ciphertext
decryptedMessage = RSADecrypt(professor_c, my_private_key)

# Convert the decrypted integer back to text 
decryptedText = intToText(decryptedMessage)
print(decryptedText)

MESSAGE FOR: Jack Salinas. Your secret number is 314. Respond with your full name and triple your secret number.


In [58]:
# Testing with professor
# Professor public key
professor_public_key = 119078858153679354996344178598913713229782582178469832460238538167861335297111373349019304311762284011809752423573709152935652133753017457925218316175121590202073053065196033751277568328455126431610393086843580003823063519848536057441797135759194647672853445004921566247396177442849332622399896197396028599661, 50802507419814693908601033314270944264944115753805663341674855772717250720680463507847236881441962534097021598426855512247397207610027891726223346928281660631173588332269779052379157358536123436642422929750025637544945896203227300698368709275639230255372802993510974013282099170446409677954392258177980964407

# Message to send
message = "Jack Salinas 942"

# Convert the message text to an integer
message_int = textToInt(message)

# Encrypt message to send
encrypted_message = RSAEncrypt(message_int, professor_public_key)
print(encrypted_message)

24417318698315865785697712706333419012814817225138891735509857470556789345031212588856098065116591988544937975138040568553952988132787982945616789728205506594983475134008050480741656478386025843495086744408681874400138880554382389069549263620317928809046853404870590270468898357674446832989384451412360366304
